In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [19]:
### Hyperparamets babyyy, change accordingly

Channel_uses = 10
Epochs = 1000
Noise_variance = 1e-2
Pert_variance = 1e-2

In [20]:
def policy_loss(w):
    t   = keras.backend.sum(keras.backend.dot(w,w))
    t   = keras.backend.exp(-t/Pert_variance**2)/(np.pi*Pert_variance**2)**Channel_uses
    loss= keras.backend.dot(l_hat,keras.backend.log(t))
    return t

In [21]:
def tx_loss(y_true, y_pred):
    t = policy_loss(y_true - y_pred)
    return 
    


In [22]:
def perturbation(d):
    W = tf.keras.backend.random_normal(shape = (2*Channel_uses,),
    mean=0.0,stddev=Pert_variance**0.5,dtype=None,seed=None)
    d = ((1-Pert_variance)**0.5)*d + W
    return d

def noise_addition(d):
    d = d + tf.keras.backend.random_normal(
    shape = (2*Channel_uses,),
    mean=0.0,
    stddev=Noise_variance**0.5,
    dtype=None,
    seed=None)
    return d

def receiver_model(d):
    #x = keras.layers.Input(shape=(2*Channel_uses))
    x = keras.layers.Lambda(noise_addition)(d)
    x = keras.layers.Dense(2*Channel_uses, activation='elu')(x)
    x = keras.layers.Dense(10*Channel_uses, activation='elu')(x)
    pred = keras.layers.Dense(1, activation='elu')(x)
    return pred


def transmitter_model(d):
    x = keras.layers.Dense(10*Channel_uses, activation='relu')(d)
    x = keras.layers.Dense(2*Channel_uses, activation='relu')(x)
    xp = keras.layers.Lambda(perturbation)(x)
    return xp


###  Tx model

In [32]:
# tx layers
tx_input = keras.layers.Input((1,), name='tx_input')
x = keras.layers.Dense(units=10*Channel_uses, activation='relu', name='tx_10')(tx_input)
x = keras.layers.Dense(units=2*Channel_uses, activation='relu', name='tx_out')(x)
#xp = keras.layers.Lambda(perturbation, name='Xp')(x)

In [33]:
tx_model = keras.models.Model(inputs=tx_input, outputs=x)

In [34]:
tx_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tx_input (InputLayer)        (None, 1)                 0         
_________________________________________________________________
tx_10 (Dense)                (None, 100)               200       
_________________________________________________________________
tx_out (Dense)               (None, 20)                2020      
Total params: 2,220
Trainable params: 2,220
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tx_model.compile(loss=tx_loss, optimizer='sgd', metrics=['accuracy'])

### Rx model

In [10]:
rx_input = keras.layers.Input((2*Channel_uses,), name='rx_input')
# channel layer
y = keras.layers.Lambda(lambda x: x+keras.backend.random_normal(
        shape = (2*Channel_uses,), mean=0.0, stddev=Noise_variance**0.5), name='channel_output')(rx_input)

y = keras.layers.Dense(2*Channel_uses, activation='elu', name='rx_2')(y)
y = keras.layers.Dense(10*Channel_uses, activation='elu', name='rx_10')(y)
pred = keras.layers.Dense(1, activation='elu', name='rx_output')(y)

In [11]:
rx_model = keras.models.Model(inputs=rx_input, outputs=pred)
rx_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rx_input (InputLayer)        (None, 20)                0         
_________________________________________________________________
channel_output (Lambda)      (None, 20)                0         
_________________________________________________________________
rx_2 (Dense)                 (None, 20)                420       
_________________________________________________________________
rx_10 (Dense)                (None, 100)               2100      
_________________________________________________________________
rx_output (Dense)            (None, 1)                 101       
Total params: 2,621
Trainable params: 2,621
Non-trainable params: 0
_________________________________________________________________


In [13]:
rx_model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

### Training


In [14]:
x_train = np.random.uniform(0,1, 10)

In [15]:
# training
XP = tx_model.predict(x_train)
r_pred = rx_model.predict(XP).flatten()
l = (x_train-r_pred)**2
l_hat = rx_model.predict(tx_model.predict(l))

In [16]:
tx_model.fit(x=x_train, XP, batch_size=None)

Epoch 1/1


InvalidArgumentError: Matrix size-incompatible: In[0]: [10,20], In[1]: [10,20]
	 [[{{node loss/tx_out_loss/MatMul}} = MatMul[T=DT_FLOAT, _class=["loc:@training/SGD/gradients/AddN"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/tx_out_loss/sub, loss/tx_out_loss/sub)]]
	 [[{{node metrics/acc/Mean/_67}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_230_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]